Практическая работа 2. Обнаружение атак на веб-приложения с применением
обучения с подкреплением

Цель работы: Освоить методы обнаружения и предотвращения атак на
веб-приложения (SQL-инъекции, XSS, DDoS) с использованием обучения с
подкреплением (Reinforcement Learning, RL).


Задание:

Изучить основные типы атак на веб-приложения: SQL-инъекции, XSS, DDoS.
Подготовить датасет с примерами нормального и аномального трафика. Разработать
среду RL на основе OpenAI Gym для моделирования веб-трафика.Реализовать агента
обучения с подкреплением для автоматической фильтрации атакующего трафика.
Провести обучение агента и оценить его эффективность.

In [2]:
import pandas as pd  # Импортируем библиотеку pandas для работы с данными
from sklearn.model_selection import train_test_split  # Импортируем функцию для разделения данных на обучающую и тестовую выборки
from sklearn.preprocessing import StandardScaler  # Импортируем стандартный скейлер для нормализации данных

# Загрузим датасет из CSV файла
data = pd.read_csv('/content/ddos.csv')  # Читаем данные из файла 'ddos.csv' и сохраняем в переменной data
data.head()  # Выводим первые 5 строк датасета для предварительного просмотра


,packet_rate,inter_arrival_time,packet_size,duration,src_port,dst_port,num_packets,num_bytes,protocol,src_ip,dst_ip,label
0,32.052620,0.622677,419.336437,4.460177,8801,9250,73.206250,7068.070150,ICMP,53.110.216.129,177.21.88.93,attack
1,52.933578,0.492452,846.565643,4.435679,3708,51281,38.226238,3848.110143,TCP,218.27.101.33,188.53.205.189,normal
2,96.720151,0.185046,502.168275,4.431496,15306,6832,32.990155,7600.730151,TCP,70.115.47.70,115.8.160.190,attack
3,80.637373,0.137000,462.529046,4.675813,48304,9779,71.610137,9700.654230,ICMP,173.214.29.104,112.116.204.53,attack
4,29.855090,0.601742,785.058163,5.295717,32704,65071,43.491605,4619.087460,TCP,120.227.151.189,201.71.89.46,normal


Предварительно обрабатываем данные, преобразуя их в формат, пригодный для обучения.

In [3]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder  # Импортируем MinMaxScaler для нормализации данных и LabelEncoder для кодирования меток

label_encoder = LabelEncoder()  # Создаем экземпляр класса LabelEncoder для преобразования категориальных меток в числовые значения

# Кодируем столбец 'protocol' в числовые значения
data['protocol'] = label_encoder.fit_transform(data['protocol'])  # Применяем кодирование к столбцу 'protocol' и сохраняем результат обратно в этот столбец


In [4]:
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

In [5]:
# Удаляем столбцы 'src_ip' и 'dst_ip' из датафрейма
data.drop(['src_ip', 'dst_ip'], axis=1, inplace=True)  # Указываем axis=1 для удаления столбцов и inplace=True для изменения датафрейма на месте

# Проверяем, что столбцы удалены, и выводим оставшиеся столбцы
print("Оставшиеся столбцы:\n", data.columns)  # Выводим названия оставшихся столбцов в датафрейме


Оставшиеся столбцы:
 Index(['packet_rate', 'inter_arrival_time', 'packet_size', 'duration',
       'src_port', 'dst_port', 'num_packets', 'num_bytes', 'protocol',
       'label'],
      dtype='object')


In [6]:
data.head()

,packet_rate,inter_arrival_time,packet_size,duration,src_port,dst_port,num_packets,num_bytes,protocol,label
0,32.052620,0.622677,419.336437,4.460177,8801,9250,73.206250,7068.070150,0,0
1,52.933578,0.492452,846.565643,4.435679,3708,51281,38.226238,3848.110143,1,1
2,96.720151,0.185046,502.168275,4.431496,15306,6832,32.990155,7600.730151,1,0
3,80.637373,0.137000,462.529046,4.675813,48304,9779,71.610137,9700.654230,0,0
4,29.855090,0.601742,785.058163,5.295717,32704,65071,43.491605,4619.087460,1,1


In [7]:
# Нормализуем числовые признаки с помощью MinMaxScaler
scaler = MinMaxScaler()  # Создаем экземпляр MinMaxScaler для нормализации данных
data[['packet_rate', 'inter_arrival_time', 'packet_size', 'duration', 'num_packets', 'num_bytes']] = scaler.fit_transform(
    data[['packet_rate', 'inter_arrival_time', 'packet_size', 'duration', 'num_packets', 'num_bytes']]
)  # Применяем нормализацию к указанным числовым признакам и сохраняем результат обратно в датафрейм

# Разделяем данные на признаки (X) и метки (y)
X = data.drop('label', axis=1)  # Убираем столбец 'label' из датафрейма для получения признаков
y = data['label'].apply(lambda x: 1 if x == 'attack' else 0)  # Преобразуем метки в бинарный формат: 1 для 'attack', 0 для остальных

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Делим данные на 80% для обучения и 20% для тестирования


In [9]:
import pandas as pd  # Импортируем библиотеку pandas для работы с данными в формате таблиц
import numpy as np  # Импортируем библиотеку NumPy для работы с массивами и математическими операциями
import random  # Импортируем модуль random для генерации случайных чисел
import gym  # Импортируем библиотеку gym для создания и работы со средами обучения с подкреплением
import torch  # Импортируем библиотеку PyTorch для работы с нейронными сетями и тензорами
import torch.nn as nn  # Импортируем модуль nn из PyTorch для создания нейронных сетей
import torch.optim as optim  # Импортируем модуль optim для работы с оптимизаторами
from tqdm import trange  # Импортируем trange из библиотеки tqdm для отображения прогресс-бара
from gym import spaces  # Импортируем пространство действий и наблюдений из gym
from sklearn.model_selection import train_test_split  # Импортируем функцию для разделения данных на обучающую и тестовую выборки
from sklearn.preprocessing import StandardScaler  # Импортируем стандартный скейлер для нормализации


In [10]:
# Создание среды для обучения с подкреплением (RL)
class TrafficEnv(gym.Env):
    def __init__(self, X, y):
        super(TrafficEnv, self).__init__()  # Инициализация базового класса gym.Env
        self.X = X  # Матрица признаков (входные данные)
        self.y = y  # Вектор меток (0 или 1, целевая переменная)
        self.current_index = 0  # Индекс текущего состояния
        # Определяем пространство наблюдений (признаки)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X.shape[1],), dtype=np.float32)
        # Определяем пространство действий (доступные действия)
        self.action_space = spaces.Discrete(2)  # 0 = пропустить, 1 = заблокировать

    # Сброс среды к начальному состоянию
    def reset(self):
        self.current_index = 0  # Сбрасываем индекс текущего состояния
        return self.X[self.current_index]  # Возвращаем начальное состояние

    def step(self, action):
        true_label = self.y[self.current_index]  # Получаем истинную метку для текущего состояния

        # Награда: +1 за правильное действие, -1 за неправильное
        if action == true_label:
            reward = 1  # Правильное действие
        else:
            reward = -1  # Неправильное действие

        self.current_index += 1  # Переходим к следующему состоянию
        done = self.current_index >= len(self.X)  # Проверяем, достигли ли конца данных

        if not done:
            next_state = self.X[self.current_index]  # Получаем следующее состояние
        else:
            next_state = np.zeros_like(self.X[0])  # Пустое состояние в конце

        return next_state, reward, done, {}  # Возвращаем следующее состояние, награду, флаг завершения и дополнительную информацию

    def render(self, mode="human"):
        pass  # Метод для визуализации среды (пока не реализован)


In [11]:
# Определение нейросети (DQN)
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()  # Инициализация базового класса nn.Module
        self.model = nn.Sequential(  # Создаем последовательную модель
            nn.Linear(input_dim, 128),  # Полносвязный слой с 128 нейронами
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(128, 64),  # Второй полносвязный слой с 64 нейронами
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(64, output_dim)  # Выходной слой (2 нейрона — Q-значения для действий)
        )

    def forward(self, x):
        return self.model(x)  # Прямое распространение через модель

# Обучение DQN-агента
# Выводим параметры
state_dim = X_train.shape[1]  # Размерность состояния (количество признаков)
action_dim = 2  # Количество действий (0 = пропустить, 1 = заблокировать)
gamma = 0.99  # Коэффициент дисконтирования для будущих наград
epsilon = 1.0  # Начальное значение ε для ε-greedy стратегии
epsilon_min = 0.01  # Минимальное значение ε
epsilon_decay = 0.995  # Скорость уменьшения ε
lr = 0.001  # Скорость обучения (learning rate)
batch_size = 64  # Размер батча для обучения
replay_memory = deque(maxlen=10000)  # Буфер воспроизведения (replay memory) с максимальной длиной 10000

# Инициализация
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Определяем устройство (GPU или CPU)
policy_net = DQN(state_dim, action_dim).to(device)  # Создаем экземпляр DQN и отправляем на устройство
optimizer = optim.Adam(policy_net.parameters(), lr=lr)  # Инициализируем оптимизатор Adam
loss_fn = nn.MSELoss()  # Функция потерь (среднеквадратичная ошибка)

# Функция действия (ε-greedy)
def select_action(state, epsilon):
    # Выбор действия (ε-greedy):
    # С вероятностью ε — случайное действие
    # Иначе — действие с максимальным Q-значением

    if np.random.rand() < epsilon:  # Случайный выбор действия с вероятностью ε
        return random.randint(0, action_dim - 1)  # Возвращаем случайное действие
    state = torch.FloatTensor(state).unsqueeze(0).to(device)  # Преобразуем состояние в тензор и отправляем на устройство
    with torch.no_grad():  # Отключаем градиенты для вычисления Q-значений
        q_values = policy_net(state)  # Получаем Q-значения для текущего состояния
    return torch.argmax(q_values).item()  # Возвращаем действие с максимальным Q-значением


In [12]:
# Создаем экземпляр среды для обучения с подкреплением, передавая матрицу признаков и вектор меток
env = TrafficEnv(X_train, y_train.values)  # Инициализация среды TrafficEnv с обучающими данными

num_episodes = 100  # Устанавливаем количество эпизодов для обучения агента
epsilon = 1.0  # Начальное значение ε для ε-greedy стратегии (максимальная вероятность случайного выбора действия)


In [ ]:
max_steps = 15000  # Максимальное количество шагов в одном эпизоде

# Цикл по количеству эпизодов обучения
for episode in trange(num_episodes, desc="Обучение агента"):
    state = env.reset()  # Сбрасываем среду и получаем начальное состояние
    total_reward = 0  # Общая награда за эпизод
    done = False  # Флаг завершения эпизода
    step_count = 0  # Счетчик шагов

    # Цикл, выполняющийся, пока эпизод не завершен и не достигнуто максимальное количество шагов
    while not done and step_count < max_steps:
        action = select_action(state, epsilon)  # Выбираем действие с помощью ε-greedy стратегии
        next_state, reward, done, _ = env.step(action)  # Выполняем действие и получаем следующее состояние, награду и флаг завершения
        total_reward += reward  # Обновляем общую награду
        step_count += 1  # Увеличиваем счетчик шагов

        # Сохраняем опыт в буфер воспроизведения
        replay_memory.append((state, action, reward, next_state, done))
        state = next_state  # Переходим к следующему состоянию

        # Обучение модели из буфера воспроизведения, если достаточно данных
        if len(replay_memory) >= batch_size:
            batch = random.sample(replay_memory, batch_size)  # Случайная выборка из буфера
            states, actions, rewards, next_states, dones = zip(*batch)  # Распаковываем выборку

            # Преобразуем данные в массивы NumPy
            states = np.array(states)
            next_states = np.array(next_states)
            actions = np.array(actions)
            rewards = np.array(rewards)
            dones = np.array(dones)

            # Преобразуем данные в тензоры PyTorch и отправляем на устройство
            states = torch.FloatTensor(states).to(device)
            actions = torch.LongTensor(actions).unsqueeze(1).to(device)
            rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
            next_states = torch.FloatTensor(next_states).to(device)
            dones = torch.BoolTensor(dones).unsqueeze(1).to(device)

            # Вычисляем Q-значения и обновляем модель
            q_values = policy_net(states).gather(1, actions)  # Получаем Q-значения для выбранных действий
            next_q_values = policy_net(next_states).max(1)[0].unsqueeze(1)  # Получаем максимальные Q-значения для следующих состояний
            expected_q = rewards + gamma * next_q_values * (~dones)  # Вычисляем ожидаемые Q-значения

            loss = loss_fn(q_values, expected_q.detach())  # Вычисляем функцию потерь
            optimizer.zero_grad()  # Обнуляем градиенты
            loss.backward()  # Обратное распространение ошибки
            optimizer.step()  # Обновляем параметры модели

    # Обновляем значение epsilon для ε-greedy стратегии
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    # Выводим информацию о текущем эпизоде
    print(f"Эпизод {episode + 1}: Общая награда = {total_reward}, шагов: {step_count}, ε = {epsilon:.3f}")


Обучение агента:   1%|          | 1/100 [00:40<1:07:07, 40.68s/it]

Эпизод 1: Общая награда = -24, шагов: 15000, ε = 0.995


Обучение агента:   2%|▏         | 2/100 [01:21<1:06:14, 40.55s/it]

Эпизод 2: Общая награда = -202, шагов: 15000, ε = 0.990


Обучение агента:   3%|▎         | 3/100 [02:00<1:04:44, 40.04s/it]

...

Эпизод 98: Общая награда = 5764, шагов: 15000, ε = 0.612


Обучение агента:  99%|█████████▉| 99/100 [1:18:42<00:48, 48.72s/it]

Эпизод 99: Общая награда = 5902, шагов: 15000, ε = 0.609


Обучение агента: 100%|██████████| 100/100 [1:19:32<00:00, 47.73s/it]

Эпизод 100: Общая награда = 5888, шагов: 15000, ε = 0.606


In [ ]:
# Переводим X_test в numpy
X_test_np = np.array(X_test)  # Преобразуем тестовые данные в массив NumPy
y_test_np = np.array(y_test)  # Преобразуем вектор меток тестовых данных в массив NumPy

# Получим предсказания от агента
predictions = []  # Список для хранения предсказанных действий

with torch.no_grad():  # Отключаем вычисление градиентов для экономии памяти
    for state in X_test_np:  # Проходим по каждому состоянию в тестовом наборе
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)  # Преобразуем состояние в тензор и отправляем на устройство
        q_values = policy_net(state_tensor)  # Получаем Q-значения от нейросети
        action = torch.argmax(q_values).item()  # Выбираем действие с максимальным Q-значением
        predictions.append(action)  # Добавляем предсказанное действие в список

# Метрики
print("Classification Report:")  # Выводим отчет о классификации
print(classification_report(y_test_np, predictions, target_names=["Benign", "Attack"]))  # Печатаем отчет с метриками для классов

print("Confusion Matrix:")  # Выводим матрицу ошибок
print(confusion_matrix(y_test_np, predictions))  # Печатаем матрицу ошибок для сравнения истинных и предсказанных меток


Classification Report:
              precision    recall  f1-score   support

      Benign       0.82      0.49      0.62      4020
      Attack       0.08      0.30      0.13       614

    accuracy                           0.47      4634
   macro avg       0.45      0.40      0.37      4634
weighted avg       0.72      0.47      0.55      4634

Confusion Matrix:
[[1980 2040]
 [ 431  183]]


In [ ]:
# Оценка модели
def evaluate_model(model, X, y):
    model.eval()  # Убедимся, что модель в режиме оценки (выключаем режим обучения)
    # Получаем предсказания для каждого состояния в X
    predictions = [np.argmax(model(torch.tensor(np.array([x]), dtype=torch.float32)).detach().numpy()) for x in X]
    # Выводим отчет о классификации, сравнивая истинные метки и предсказанные
    print(classification_report(y, predictions))

# Вызываем функцию оценки модели, передавая нейросеть и тестовые данные
evaluate_model(policy_net, X_test, y_test)


              precision    recall  f1-score   support

           0       0.82      0.49      0.62      4020
           1       0.08      0.30      0.13       614

    accuracy                           0.47      4634
   macro avg       0.45      0.40      0.37      4634
weighted avg       0.72      0.47      0.55      4634



In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Импортируем класс RandomForestClassifier из библиотеки scikit-learn
from sklearn.svm import SVC  # Импортируем класс SVC (Support Vector Classifier) из библиотеки scikit-learn

# Сравнение с классическими методами
rf = RandomForestClassifier()  # Создаем экземпляр классификатора Random Forest
rf.fit(X_train, y_train)  # Обучаем модель на обучающих данных
y_pred_rf = rf.predict(X_test)  # Получаем предсказания для тестовых данных
print("Random Forest:")  # Выводим заголовок для результатов Random Forest
print(classification_report(y_test, y_pred_rf))  # Печатаем отчет о классификации для Random Forest

svm = SVC()  # Создаем экземпляр классификатора SVM
svm.fit(X_train, y_train)  # Обучаем модель на обучающих данных
y_pred_svm = svm.predict(X_test)  # Получаем предсказания для тестовых данных
print("SVM:")  # Выводим заголовок для результатов SVM
print(classification_report(y_test, y_pred_svm))  # Печатаем отчет о классификации для SVM


Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4020
           1       1.00      1.00      1.00       614

    accuracy                           1.00      4634
   macro avg       1.00      1.00      1.00      4634
weighted avg       1.00      1.00      1.00      4634

SVM:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4020
           1       0.99      0.95      0.97       614

    accuracy                           0.99      4634
   macro avg       0.99      0.97      0.98      4634
weighted avg       0.99      0.99      0.99      4634

